In [94]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
import CSV
using JSON
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

In [95]:
includet(pwd() * "\\Water_Regulation\\WaterRegulation.jl")
using .WaterRegulation

### General Parameters

In [96]:
filepath_Ljungan = pwd() * "\\Water_Regulation\\TestDataWaterRegulation\\Ljungan.json"
filepath_prices = pwd() *  "\\Data\\Spot Prices\\prices_df.csv"
filepath_results = pwd() * "\\Results\\LambdaZero\\"
all_res, plants, parts = read_data(filepath_Ljungan)
print() 

### River System  
We will work with Sydkraft in this example  
For ease at first we have one producer, one reservoir and one plant.

In [97]:
j = parts[1]
res = filter(r -> j.participationrate[r] > 0, all_res)[1]
k = j.plants[1]

println("Participant: ", j)
println("Reservoir: ", res)
println("Power Plant: ", k)

Participant: Sydkraft

Reservoir: Flasjon

Power Plant: Flasjo



### Bidding Model  
At the Electricity Markets, Participants have to communicate increasing bidding curves of price-volume pairs before market clearing.  
Depending on the market clearing price, the delivery of each participant becomes known  
We approximate this relation by using linear interpolation of volumes and presetting price Points, for example based on probabilities.

$$
y_t = \frac{c_t - P_{t,i} }{P_{t,i+1} - P_{t,i}} \cdot x_{i,t} + \frac{P_{t,i+1} - c_t }{P_{t,i+1} - P{t,i}} \cdot x_{i+1,t}, \qquad \text{if} \qquad P_{t,i} \leq c_t \leq P_{t,i+1}
$$

The Volumes have to be in increasing order:

$$
x_{i,t} \leq x_{i+1,t}
$$

### Hydro Model

The Commodity we deal with is electricity. At all times it has to be in balance and consumed at the moment it is generated. The means of producing electricity is a pure hydro system with a reservoir upstream for storage.  

$$
\begin{align*}
l &= \text{Reservoir level} \\
Q &= \text{Flow of Water} \\
w &= \text{Electrical Production} \\
e &= \text{Equivalent}
\end{align*}
$$

### Set Parameters necessary for Input into Model

In [98]:
Stages = 8 # 2 lowest number as first stage is just to achieve nonanticipativity
T = 24
PPoints = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
I = length(PPoints)-1

println("Stages :" , Stages, "\n Hours: ", T, "\n I : ", I, "\n Price Points = ", [i for i in PPoints])

Qref = Dict{Reservoir, Float64}(r => 10.0 for r in [res])
scenario_count = 2
Prices = [floor.(rand(T), sigdigits=3) for i in 1:scenario_count]
Inflows = [10.0]
Omega = [(price = p, inflow = v) for p in Prices for v in Inflows]
P = [1/length(Omega) for om in Omega]
# StartUp Costs
S = 0.1
# Cost for Up and Down Balancing
mu = 1

println("Uncertainty Set: ", Omega)

Stages :8
 Hours: 24
 I : 5
 Price Points = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]


Uncertainty Set: NamedTuple{(:price, :inflow), Tuple{Vector{Float64}, Float64}}[(price = [0.457, 0.715, 0.951, 0.239, 0.533, 0.405, 0.513, 0.121, 0.865, 0.794, 0.275, 0.0664, 0.647, 0.037, 0.93, 0.0192, 0.173, 0.599, 0.989, 0.368, 0.257, 0.342, 0.0968, 0.794], inflow = 10.0), (price = [0.295, 0.254, 0.773, 0.7, 0.189, 0.521, 0.433, 0.995, 0.411, 0.468, 0.472, 0.798, 0.776, 0.188, 0.556, 0.365, 0.0498, 0.0104, 0.502, 0.0997, 0.422, 0.694, 0.622, 0.428], inflow = 10.0)]


### Definition of Subproblems

In [106]:
function subproblem_builder(subproblem::Model, node::Int64)
    @variable(subproblem, 0 <= x[i = 1:I+1, t = 1:T] <= 50, SDDP.State, initial_value=0)
    @variable(subproblem, 0 <= l <= res.maxvolume, SDDP.State, initial_value = res.currentvolume)
    @variable(subproblem, y[i = 1:I, t=1:T] >= 0)
    @variable(subproblem, w[i = 1:I, t=1:T] >= 0)
    @variable(subproblem, b[i = 1:I, t=1:T] >= 0)
    @variable(subproblem, Q[i = 1:I, t=1:T] >= 0)
    @variable(subproblem, f >= 0)
    @variable(subproblem, s >= 0)

    @constraint(subproblem, clearing[i = 1:I,t=1:T], y[i,t] == 1* x[i,t].in +  1* x[i+1,t].in)
    @constraint(subproblem, increasing[i = 1:I, t=1:T], x[i,t].out <= x[i+1,t].out)
    if node == 1
        @stageobjective(subproblem, 0)
        @constraint(subproblem, balance_transfer, l.in == l.out)
    else
        @constraint(subproblem, obligation[i = 1:I, t=1:T], y[i,t] == w[i,t] + b[i,t])
        @constraint(subproblem, balance, l.out == l.in - sum(Q[i,t] for t in 1:T for i in 1:I) + f * T - s)
        @constraint(subproblem, production[i = 1:I, t=1:T], w[i,t] == Q[i,t] * k.equivalent)
        SDDP.parameterize(subproblem, Omega, P) do om
            # We have to make sure that depending on the market clearing price, the coefficients are set accordingly.
            # The recourse action only applies to the real delivery, determined by the uncertain price. The other restricitions become inactive, else they make the problem infeasible.
            # The constraints that are relevant are maiintained in Scenario_Index for every current time step.
            JuMP.fix(f, om.inflow, force=true)
            # Define Set of active variables for each hour
            I_t = Dict(t => 0 for t in 1:T)
            for t in 1:T
                for i in 1:I
                    if (om.price[t] >= PPoints[i]) && (om.price[t] <= PPoints[i+1])
                        I_t[t] = i
                    end
                end
            end
            # Include only active variables in stageobjective
            @stageobjective(subproblem ,sum(om.price[t] * y[I_t[t],t] -  mu * b[I_t[t],t] for t in 1:T))
            # Fix / Deactivate constraints by setting their coefficients to appropriate values or all zero.
            for t in 1:T
                for i in 1:I
                    if (i == I_t[t])
                        set_normalized_coefficient(clearing[i,t], y[i,t], 1)
                        set_normalized_coefficient(clearing[i,t], x[i,t].in, -((om.price[t] - PPoints[i])/(PPoints[i+1] - PPoints[i])))
                        set_normalized_coefficient(clearing[i,t], x[i+1,t].in, -((PPoints[i+1] - om.price[t])/(PPoints[i+1] - PPoints[i])))
                        set_normalized_coefficient(balance, Q[i,t], 1)
                        set_normalized_coefficient(obligation[i,t], w[i,t], -1)
                        set_normalized_coefficient(obligation[i,t], b[i,t], -1)
                        set_normalized_coefficient(obligation[i,t], y[i,t], 1)
                        set_normalized_coefficient(production[i,t], w[i,t], 1)
                        set_normalized_coefficient(production[i,t], Q[i,t], -k.equivalent)
                    else
                        set_normalized_coefficient(balance, Q[i,t], 0)
                        set_normalized_coefficient(obligation[i,t], w[i,t], 0)
                        set_normalized_coefficient(obligation[i,t], b[i,t], 0)
                        set_normalized_coefficient(obligation[i,t], y[i,t], 0)
                        set_normalized_coefficient(clearing[i,t], y[i,t], 0)
                        set_normalized_coefficient(clearing[i,t], x[i,t].in, 0)
                        set_normalized_coefficient(clearing[i,t], x[i+1,t].in, 0)
                        set_normalized_coefficient(production[i,t], w[i,t], 0)
                        set_normalized_coefficient(production[i,t], Q[i,t], 0)
                    end
                end
            end
        end
    end
    return
end

subproblem_builder (generic function with 1 method)

In [107]:
for i in 1:I
    println(PricePoints[i])
end
for el in Omega
    println(el)
end


0.0
1.0


BoundsError: BoundsError: attempt to access 2-element Vector{Float64} at index [3]

### Build the SDDP Model

In [113]:
model = SDDP.LinearPolicyGraph(
    subproblem_builder;
    stages = Stages,
    sense = :Max,
    upper_bound = 1e5,
    optimizer = CPLEX.Optimizer
)

A policy graph with 8 nodes.
 Node indices: 1, 2, 3, 4, 5, 6, 7, 8


### Train the model

In [114]:
SDDP.train(model; iteration_limit = 100)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 8
  state variables : 145
  scenarios       : 1.28000e+02
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [773, 773]
  VariableRef in MOI.LessThan{Float64}    : [146, 146]
  AffExpr in MOI.LessThan{Float64}        : [120, 120]
  VariableRef in MOI.GreaterThan{Float64} : [627, 628]
  AffExpr in MOI.EqualTo{Float64}         : [121, 361]
numerical stability report
  matrix range     [5e-03, 2e+01]
  objective range  [1e-02, 1e+00]
  bounds range     [5e+01, 1e+05]
  rhs range        [0e+00, 0e+00]
-------------------------------------------------------------------
 iteration    simulation      bound       

         1   3.605107e+02  1.287057e+04  1.199999e-01        23   1
         2  -2.778155e+03  3.886453e+03  1.970000e-01        46   1


         3  -6.582450e+02  3.886453e+03  2.760000e-01        69   1


         4   2.447022e+03  3.886453e+03  4.240000e-01        92   1
         5   2.794037e+03  3.886453e+03  5.160000e-01       115   1


         6   2.790555e+03  3.886453e+03  5.899999e-01       138   1
         7   2.873158e+03  3.886453e+03  6.699998e-01       161   1


...

        96   3.882340e+03  3.886452e+03  9.861000e+00      2208   1


        97   3.890565e+03  3.886452e+03  9.969000e+00      2231   1


        98   3.890565e+03  3.886452e+03  1.007900e+01      2254   1


        99   3.898790e+03  3.886452e+03  1.019400e+01      2277   1


       100   3.898790e+03  3.886452e+03  1.031100e+01      2300   1
-------------------------------------------------------------------
status         : iteration_limit
total time (s) : 1.031100e+01
total solves   : 2300
best bound     :  3.886452e+03
simulation ci  :  3.652891e+03 ± 1.757748e+02
numeric issues : 0
-------------------------------------------------------------------



### Obtain the Bidding and Nomination Decisions

In [115]:
rule = SDDP.DecisionRule(model; node = 1)
solution = SDDP.evaluate(
    rule;
    incoming_state = Dict(:l => res.currentvolume),
    #incoming_state = merge(Dict(Symbol("l_real[$(r)]") => r.currentvolume for r in res), Dict(Symbol("l_ind[$(r)]") => r.currentvolume for r in res)),
    controls_to_record = [:Q, :y, :x, :l]
)
println(solution.controls[:x])
print(solution.controls[:l])

SDDP.State{Float64}[SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State

{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0)

50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0)]


In [123]:
simulations = SDDP.simulate(
    # The trained model to simulate.
    model,
    # The number of replications.
    100,
    # A list of names to record the values of.
    [:x, :y, :w, :l, :b, :Q, :f],
)

100-element Vector{Vector{Dict{Symbol, Any}}}:
 [Dict(:b => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], :l => SDDP.State{Float64}(35000.0, 35000.0), :w => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], :bellman_term => 3886.4524999999994, :node_index => 1, :objective_state => nothing, :belief => Dict(1 => 1.0), :x => SDDP.State{Float64}[SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 0.0); … ; SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0); SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0) … SDDP.State{Float64}(0.0, 50.0) SDDP.State{Float64}(0.0, 50.0)], :stage_objective => 0, :Q => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0

In [132]:
replication = 28
stage = 8
println("Bidding Curve Values: ", simulations[replication][stage][:x])
println("Cleared Volume: ", simulations[replication][stage][:y])
println("Reservoir Level: ", simulations[replication][stage][:l])
println("Own Production: ", simulations[replication][stage][:w])
println("Water Used: ", simulations[replication][stage][:Q])
println("Purchases: ", simulations[replication][stage][:b])
println("Filling in that stage: ", simulations[replication][stage][:f])

Bidding Curve Values: SDDP.State{Float64}[SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0); SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State

{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0); SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(0.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 0.0) SDDP.State{Float64}(50.0, 

Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0); SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64}(50.0, 50.0) SDDP.State{Float64

### Inclusion of Water Regulation Rules

The Water Regulation Rules have to be accounted for when bidding at the Market

* We communicate a sequence of bids at the electricity markets for every hour of the day
* The Water Regulation company receives ONE Bid for discharge over the entire day

For currrent purposes, we assume that the nomination is fixed and can't be changed, and the amount of water has to actually be used over the course of the day.  

* We have to buy electricity on the market for certain hours if our nomination was too low
* We have to sell surplus of water if our nomination was too high. The price on the balancing market is by assumption very low

For that we include an additional State Variable: $Q^\text{nom}_r$ for every reservoir. Let $Q_{r,t}$ the real flow of water from $r$ at hour $t$.   
For every reservoir it has to hold that:  

$$
T \cdot Q^\text{nom}_r = \sum\limits_{t=1}^T Q_{r,t}
$$

We change the objective Function to the following:


$$
\max \; \rightarrow \; \sum\limits_{t = 1}^T y_{i_t, t} \cdot c_t - (\mu^+ \cdot z^+_{i_t,t}  - \mu^- \cdot z^-_{i_t,t})
$$
The coefficient $\mu^+, \mu^-$ are chosen so that they are higher than the highest or lower than the lowest observed spot market price respectively. 
Accordingly, the delivery obligation constraint is changed to (assuming there is one river and one power plant): 

$$
y_{i_t, t} = Q_{r,t} \cdot e_k + z^+_{t} - z^-_{t}  
$$

The difficulty in the agents decision is that the nomination $Q^\text{nom}_r$ has to be done before market clearing, thus it is modelled as a state variable.

In [ ]:
function subproblem_builder_regulation(subproblem::Model, node::Int64)
    @variable(subproblem, 0 <= x[i = 1:I+1, t = 1:T] <= 50, SDDP.State, initial_value=0)
    @variable(subproblem, 0 <= l <= res.maxvolume, SDDP.State, initial_value = res.currentvolume)
    @variable(subproblem, 0 <= )
    @variable(subproblem, y[i = 1:I, t=1:T] >= 0)
    @variable(subproblem, w[i = 1:I, t=1:T] >= 0)
    @variable(subproblem, b[i = 1:I, t=1:T] >= 0)
    @variable(subproblem, Q[i = 1:I, t=1:T] >= 0)
    @variable(subproblem, f >= 0)
    @variable(subproblem, s >= 0)

    @constraint(subproblem, clearing[i = 1:I,t=1:T], y[i,t] == 1* x[i,t].in +  1* x[i+1,t].in)
    @constraint(subproblem, increasing[i = 1:I, t=1:T], x[i,t].out <= x[i+1,t].out)
    if node == 1
        @stageobjective(subproblem, 0)
        @constraint(subproblem, balance_transfer, l.in == l.out)
    else
        @constraint(subproblem, obligation[i = 1:I, t=1:T], y[i,t] == w[i,t] + b[i,t])
        @constraint(subproblem, balance, l.out == l.in - sum(Q[i,t] for t in 1:T for i in 1:I) + f * T - s)
        @constraint(subproblem, production[i = 1:I, t=1:T], w[i,t] == Q[i,t] * k.equivalent)
        SDDP.parameterize(subproblem, Omega, P) do om
            # We have to make sure that depending on the market clearing price, the coefficients are set accordingly.
            # The recourse action only applies to the real delivery, determined by the uncertain price. The other restricitions become inactive, else they make the problem infeasible.
            # The constraints that are relevant are maiintained in Scenario_Index for every current time step.
            JuMP.fix(f, om.inflow, force=true)
            # Define Set of active variables for each hour
            I_t = Dict(t => 0 for t in 1:T)
            for t in 1:T
                for i in 1:I
                    if (om.price[t] >= PPoints[i]) && (om.price[t] <= PPoints[i+1])
                        I_t[t] = i
                    end
                end
            end
            # Include only active variables in stageobjective
            @stageobjective(subproblem ,sum(om.price[t] * y[I_t[t],t] -  mu * b[I_t[t],t] for t in 1:T))
            # Fix / Deactivate constraints by setting their coefficients to appropriate values or all zero.
            for t in 1:T
                for i in 1:I
                    if (i == I_t[t])
                        set_normalized_coefficient(clearing[i,t], y[i,t], 1)
                        set_normalized_coefficient(clearing[i,t], x[i,t].in, -((om.price[t] - PPoints[i])/(PPoints[i+1] - PPoints[i])))
                        set_normalized_coefficient(clearing[i,t], x[i+1,t].in, -((PPoints[i+1] - om.price[t])/(PPoints[i+1] - PPoints[i])))
                        set_normalized_coefficient(balance, Q[i,t], 1)
                        set_normalized_coefficient(obligation[i,t], w[i,t], -1)
                        set_normalized_coefficient(obligation[i,t], b[i,t], -1)
                        set_normalized_coefficient(obligation[i,t], y[i,t], 1)
                        set_normalized_coefficient(production[i,t], w[i,t], 1)
                        set_normalized_coefficient(production[i,t], Q[i,t], -k.equivalent)
                    else
                        set_normalized_coefficient(balance, Q[i,t], 0)
                        set_normalized_coefficient(obligation[i,t], w[i,t], 0)
                        set_normalized_coefficient(obligation[i,t], b[i,t], 0)
                        set_normalized_coefficient(obligation[i,t], y[i,t], 0)
                        set_normalized_coefficient(clearing[i,t], y[i,t], 0)
                        set_normalized_coefficient(clearing[i,t], x[i,t].in, 0)
                        set_normalized_coefficient(clearing[i,t], x[i+1,t].in, 0)
                        set_normalized_coefficient(production[i,t], w[i,t], 0)
                        set_normalized_coefficient(production[i,t], Q[i,t], 0)
                    end
                end
            end
        end
    end
    return
end

### Increased Complexity of Hydro System  

The system of creating electricity using hydropower can be modelled using higher complexity than a constant power coefficient. So far this does not reflect:

* Multiple Reservoirs
* Multiple Power Plants at one reservoir
* Generators at Power Plants beeing on/off
* Start-Up Costs of Reservoirs

Then we include



In [ ]:
function subproblem_builder_hydro(subproblem::Model, node::Int64)
    @variable(subproblem, 0 <= x[i = 1:I+1, t = 1:T] <= 50, SDDP.State, initial_value=0)
    @variable(subproblem, 0 <= l <= res.maxvolume, SDDP.State, initial_value = res.currentvolume)
    @variable(subproblem, y[i = 1:I, t=1:T] >= 0)
    @variable(subproblem, w[i = 1:I, t=1:T] >= 0)
    @variable(subproblem, b[i = 1:I, t=1:T] >= 0)
    @variable(subproblem, Q[i = 1:I, t=1:T] >= 0)
    @variable(subproblem, f >= 0)
    @variable(subproblem, s >= 0)

    @constraint(subproblem, clearing[i = 1:I,t=1:T], y[i,t] == 1* x[i,t].in +  1* x[i+1,t].in)
    @constraint(subproblem, increasing[i = 1:I, t=1:T], x[i,t].out <= x[i+1,t].out)
    if node == 1
        @stageobjective(subproblem, 0)
        @constraint(subproblem, balance_transfer, l.in == l.out)
    else
        @constraint(subproblem, obligation[i = 1:I, t=1:T], y[i,t] == w[i,t] + b[i,t])
        @constraint(subproblem, balance, l.out == l.in - sum(Q[i,t] for t in 1:T for i in 1:I) + f * T - s)
        @constraint(subproblem, production[i = 1:I, t=1:T], w[i,t] == Q[i,t] * k.equivalent)
        SDDP.parameterize(subproblem, Omega, P) do om
            # We have to make sure that depending on the market clearing price, the coefficients are set accordingly.
            # The recourse action only applies to the real delivery, determined by the uncertain price. The other restricitions become inactive, else they make the problem infeasible.
            # The constraints that are relevant are maiintained in Scenario_Index for every current time step.
            JuMP.fix(f, om.inflow, force=true)
            # Define Set of active variables for each hour
            I_t = Dict(t => 0 for t in 1:T)
            for t in 1:T
                for i in 1:I
                    if (om.price[t] >= PPoints[i]) && (om.price[t] <= PPoints[i+1])
                        I_t[t] = i
                    end
                end
            end
            # Include only active variables in stageobjective
            @stageobjective(subproblem ,sum(om.price[t] * y[I_t[t],t] -  mu * b[I_t[t],t] for t in 1:T))
            # Fix / Deactivate constraints by setting their coefficients to appropriate values or all zero.
            for t in 1:T
                for i in 1:I
                    if (i == I_t[t])
                        set_normalized_coefficient(clearing[i,t], y[i,t], 1)
                        set_normalized_coefficient(clearing[i,t], x[i,t].in, -((om.price[t] - PPoints[i])/(PPoints[i+1] - PPoints[i])))
                        set_normalized_coefficient(clearing[i,t], x[i+1,t].in, -((PPoints[i+1] - om.price[t])/(PPoints[i+1] - PPoints[i])))
                        set_normalized_coefficient(balance, Q[i,t], 1)
                        set_normalized_coefficient(obligation[i,t], w[i,t], -1)
                        set_normalized_coefficient(obligation[i,t], b[i,t], -1)
                        set_normalized_coefficient(obligation[i,t], y[i,t], 1)
                        set_normalized_coefficient(production[i,t], w[i,t], 1)
                        set_normalized_coefficient(production[i,t], Q[i,t], -k.equivalent)
                    else
                        set_normalized_coefficient(balance, Q[i,t], 0)
                        set_normalized_coefficient(obligation[i,t], w[i,t], 0)
                        set_normalized_coefficient(obligation[i,t], b[i,t], 0)
                        set_normalized_coefficient(obligation[i,t], y[i,t], 0)
                        set_normalized_coefficient(clearing[i,t], y[i,t], 0)
                        set_normalized_coefficient(clearing[i,t], x[i,t].in, 0)
                        set_normalized_coefficient(clearing[i,t], x[i+1,t].in, 0)
                        set_normalized_coefficient(production[i,t], w[i,t], 0)
                        set_normalized_coefficient(production[i,t], Q[i,t], 0)
                    end
                end
            end
        end
    end
    return
end